In [2]:
#@title Get text train and test ,X and Y
nrows_train = 20000 # @param {type:"integer"}
sequence_length = 13 # @param {type:"integer"}

feed_lenght = 8

import pandas as pd
import numpy as np
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import json
from keras.preprocessing.text import tokenizer_from_json

#load fusion
fus = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gpt/i+d/fusionml.csv",
                  parse_dates=["time"], index_col="time")

#Get the train and test train
texts_train = fus["fusion"].sample(nrows_train,)
texts_test = fus["fusion"].drop(texts_train.index)

#save texts test
texts_test.to_csv("/content/drive/MyDrive/Colab Notebooks/gpt/i+d/texts_testml.csv")

# Tokenize text
tokenizer = Tokenizer()

#tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(fus["fusion"])

#Save tokenizer
tokenizer_json = tokenizer.to_json()

# Save the JSON configuration to a file
with open('/content/drive/MyDrive/Colab Notebooks/gpt/i+d/tokenizerml.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

sequences = tokenizer.texts_to_sequences(texts_train)

# Prepare input and output data
X = []
y = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        x_seq = sequence[:i]
        x_seq_padded = pad_sequences([x_seq], maxlen=sequence_length, padding='pre')
        X.append(x_seq_padded[0])
        y.append(sequence[i])

X = np.array(X)
y = np.array(y)

#filter seed/model words
df = pd.DataFrame(X)
df["y"] = y
df_fil =  df[(df.iloc[:, :sequence_length-feed_lenght+1] != 0).any(axis=1)]
X = df_fil.iloc[:, :-1].values
y = df_fil.iloc[:, -1].values

# One hot encode the outputs
y = np.eye(len(tokenizer.word_index) + 1)[y]

print("Dictionary size (words): len(y[i]):",len(y[1]) )
print("Total X variables:",len(X) )

print("\nText train example1")
display(texts_train[0])

print("\nSequences example 1")
display(sequences[0])

print("\nText train example2")
display(texts_train[1])

print("\nSequences example 2")
display(sequences[1])

print("\nX")
display(X[:25])

print("\nDictionary first 10 words")
display({k: tokenizer.word_index[k] for k in list(tokenizer.word_index)[:10]})


Dictionary size (words): len(y[i]): 3117
Total X variables: 182393

Text train example1


'34021kt 9999 RA FEW025 SCT 07 05 q1018 01017G29KT 340V050 7000 -SHRA FEW014 BKN020 SCT030TCU 05/03 Q1017 TEMPO 3000 SHRA BKN014'


Sequences example 1


[1604,
 1,
 8,
 49,
 22,
 16,
 24,
 25,
 2410,
 184,
 100,
 45,
 168,
 94,
 147,
 24,
 40,
 28,
 10,
 21,
 45,
 129]


Text train example2


...

[160, 1, 2, 17, 4, 12, 6, 33, 48, 15, 12, 6, 33, 3]


X


array([[   0,    0,    0,    0,    0, 1604,    1,    8,   49,   22,   16,
          24,   25],
       [   0,    0,    0,    0, 1604,    1,    8,   49,   22,   16,   24,
          25, 2410],
       [   0,    0,    0, 1604,    1,    8,   49,   22,   16,   24,   25,
        2410,  184],
       [   0,    0, 1604,    1,    8,   49,   22,   16,   24,   25, 2410,
         184,  100],
       [   0, 1604,    1,    8,   49,   22,   16,   24,   25, 2410,  184,
         100,   45],
       [1604,    1,    8,   49,   22,   16,   24,   25, 2410,  184,  100,
          45,  168],
       [   1,    8,   49,   22,   16,   24,   25, 2410,  184,  100,   45,
         168,   94],
       [   8,   49,   22,   16,   24,   25, 2410,  184,  100,   45,  168,
          94,  147],
       [  49,   22,   16,   24,   25, 2410,  184,  100,   45,  168,   94,
         147,   24],
       [  22,   16,   24,   25, 2410,  184,  100,   45,  168,   94,  147,
          24,   40],
       [  16,   24,   25, 2410,  184,  100,   45, 


Dictionary first 10 words


{'9999': 1,
 'wm': 2,
 'nosig': 3,
 'm': 4,
 '11': 5,
 '12': 6,
 '10': 7,
 'ra': 8,
 '13': 9,
 'tempo': 10}

In [3]:
#@title Train the model

from keras.layers import Bidirectional

# Build the LSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=20, input_length=sequence_length),
    Bidirectional(LSTM(130)),

    Dense(len(tokenizer.word_index)+1, activation='softmax')
])


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=70,batch_size=512)

#save the model
model.save("/content/drive/MyDrive/Colab Notebooks/gpt/i+d/model_ml.keras")


Epoch 1/70
357/357 [==============================] - 27s 66ms/step - loss: 3.8810 - accuracy: 0.2185
Epoch 2/70
357/357 [==============================] - 23s 65ms/step - loss: 2.6541 - accuracy: 0.3320
Epoch 3/70
357/357 [==============================] - 23s 64ms/step - loss: 2.3040 - accuracy: 0.3917
Epoch 4/70
357/357 [==============================] - 23s 64ms/step - loss: 2.1503 - accuracy: 0.4186
Epoch 5/70
357/357 [==============================] - 23s 65ms/step - loss: 2.0555 - accuracy: 0.4322
Epoch 6/70
357/357 [==============================] - 23s 66ms/step - loss: 1.9797 - accuracy: 0.4452
Epoch 7/70
357/357 [==============================] - 23s 64ms/step - loss: 1.9171 - accuracy: 0.4554
Epoch 8/70
357/357 [==============================] - 23s 64ms/step - loss: 1.8535 - accuracy: 0.4665
Epoch 9/70
357/357 [==============================] - 23s 64ms/step - loss: 1.8000 - accuracy: 0.4751
Epoch 10/70
357/357 [==============================] - 23s 65ms/step - loss: 1.742